# 1/ Import Libraries

In [1]:
import numpy as np
import pandas as pd
import h5py
from pathlib import Path
from typing import Dict, Callable

from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy

# 2/ Background Information

## Calculating Mutual Information Using Histograms

### 1. **Understanding the Data and the Problem**:
Let's assume you have data for two discrete random variables \( X \) and \( Y \). This data can be thought of as samples or observations of these variables.

### 2. **Construct Histograms**:

#### a. **Joint Histogram**:
Construct a 2D histogram for \( X \) and \( Y \). In this 2D histogram:
- The x-axis represents all possible values of \( X \).
- The y-axis represents all possible values of \( Y \).
- Each cell (or bin) in this 2D histogram represents a combination of values for \( X \) and \( Y \). The count in each bin represents how often that combination occurs in your data.

#### b. **Marginal Histograms**:
Construct individual histograms for \( X \) and \( Y \):
- For \( X \): Count how often each value of \( X \) occurs in your data.
- For \( Y \): Count how often each value of \( Y \) occurs in your data.

### 3. **Normalize Histograms to Get Probabilities**:

#### a. **Joint Probabilities**:
To convert the joint histogram counts into probabilities, divide the count in each bin by the total number of data points. This will give you an estimate of the joint probability \( P(X=x, Y=y) \) for each combination of \( x \) and \( y \).

#### b. **Marginal Probabilities**:
Similarly, to get the marginal probabilities from the individual histograms:
- For \( X \): Divide each bin's count by the total number of data points to get \( P(X=x) \).
- For \( Y \): Do the same to get \( P(Y=y) \).

### 4. **Compute Mutual Information**:

Using the normalized histograms, you can calculate the MI with the following formula:


$$ I(X;Y) = \sum_{x \in X} \sum_{y \in Y} P_{X,Y}(x,y) \log \left( \frac{P_{X,Y}(x,y)}{P_X(x) P_Y(y)} \right) $$




For each bin in the joint histogram:

- Get the joint probability \( P(X=x, Y=y) \) from the normalized joint histogram.
- Get the marginal probabilities \( P(X=x) \) and \( P(Y=y) \) from the normalized marginal histograms.
- Use these values in the MI formula above to compute the contribution from this bin.
- Sum up contributions from all bins to get the final MI value.

### 5. **Interpret the Result**:

The resulting MI value quantifies the amount of information shared between \( X \) and \( Y \):
- An MI of 0 suggests \( X \) and \( Y \) are independent.
- A higher MI indicates a higher dependency between \( X \) and \( Y \).

### Points to Note:

- The precision of your histograms (number and size of bins) can affect the MI calculation.
- This method gives an estimate of MI based on empirical data. The true MI would require knowledge of the true underlying probability distributions.
- Ensure you have enough data. Sparse data can lead to unreliable MI values.
- This method is specifically for discrete variables. If you have continuous data, it needs to be discretized before this approach can be used. The way you discretize can also impact the result.


# 3/ Function Definition

In [2]:
def read_h5_to_dataframe(h5_path: Path) -> pd.DataFrame:
    """
    Read a .h5 file and convert all data into a pandas dataframe.
    param h5_path: Path to location of .h5 file
    return: a pandas dataframe
    """
    data = {}
    
    with h5py.File(h5_path, 'r') as file:
        keys = list(file.keys())
        for key in keys:
            data[key] = file[key][:]
    
    return pd.DataFrame(data)


def mutual_information_binary(X: pd.Series, Y: pd.Series) -> np.float64:
    """
    A function to calculate the mutual information between two discrete variables,
    assuming the two variables X and Y are binary or can be categorized into two groups.
    This approach using normalized histograms to estimate the joint and marginal distributions.
    param X: first variable
    param Y: second variable
    return: mutual information between X and Y
    """
    joint_prob = np.histogram2d(X, Y, bins=2)[0] / len(X)
    prob_X = np.histogram(X, bins=2)[0] / len(X)
    prob_Y = np.histogram(Y, bins=2)[0] / len(Y)

    mi = 0
    for i in range(2):
        for j in range(2):
            if joint_prob[i][j] > 0:
                mi += joint_prob[i][j] * np.log(joint_prob[i][j] / (prob_X[i] * prob_Y[j]))

    return mi


def mutual_information_multiple_discrete(X: pd.Series, Y: pd.Series) -> np.float64:
    """
    A function to calculate the mutual information between two discrete variables,
    assuming the two variables X and Y have multiple discrete values.
    This approach using normalized histograms to estimate the joint and marginal distributions.
    The sum of the joint histogram may not neccessarily be 1.
    param X: first variable
    param Y: second variable
    return: mutual information between X and Y
    """
    joint_prob, x_edges, y_edges = np.histogram2d(X, Y, bins=(len(set(X)), len(set(Y))), density=True)
    prob_X = np.histogram(X, bins=len(set(X)), density=True)[0]
    prob_Y = np.histogram(Y, bins=len(set(Y)), density=True)[0]

    mi = 0
    for i in range(len(set(X))):
        for j in range(len(set(Y))):
            if joint_prob[i][j] > 0:
                mi += joint_prob[i][j] * np.log(joint_prob[i][j] / (prob_X[i] * prob_Y[j]))

    return mi


def mutual_info_with_entropy(X: pd.Series, Y: pd.Series) -> np.float64:
    """
    A function to calculate the mutual information between two discrete variables.
    This approach first calculate the entropies of each individual variable and their join entropy.
    The mutual information is then obtained using the relation: MI(X, Y) = H(X) + H(Y) - H(X, Y)
    The sum of the joint histogram may not neccessarily be 1.
    param X: first variable
    param Y: second variable
    return: mutual information between X and Y
    """
    # Calculate the individual entropies
    h_x = entropy(np.histogram(X, bins=len(set(X)))[0] / len(X))
    h_y = entropy(np.histogram(Y, bins=len(set(Y)))[0] / len(Y))

    # Calculate the joint histogram and joint entropy
    c_xy = np.histogram2d(X, Y, bins=(len(set(X)), len(set(Y))))[0]
    h_xy = entropy(c_xy.reshape(-1))

    # Compute the mutual information
    mutual_info = h_x + h_y - h_xy
    return mutual_info


def compute_mi(data: pd.DataFrame, mi_func: Callable) -> Dict:
    """
    This function compute the mutual information of each pair of variables in the dataset.
    The results are stored in an dictionary.
    param data: an input dataframe containing variables' values 
    param mi_func: a function to calculate the mutual information
    return: a dictionary containing mutual information of all variable pairs.
    """
    results = {}
    vars = data.columns
    for i in range(len(vars)):
        for j in range(i + 1, len(vars)):
            var1 = vars[i]
            var2 = vars[j]
            try:
                mi = mi_func(data[var1], data[var2])
            except ValueError:
                mi = mi_func(np.array(data[var1]).reshape(-1, 1), data[var2], discrete_features=True)
            results[(var1, var2)] = mi
    
    # Display results
    for var, mi_value in results.items():
        print(f"Mutual Information between {var[0]} and {var[1]}: {mi_value}")

    return results


# 4/ Read and convert .h5 file to a Pandas Dataframe

In [3]:
df = read_h5_to_dataframe(h5_path="../data/raw/question2-data.h5")
df.head()

,a,b,x,y
0,0,0,1,1
1,0,0,-1,-1
2,1,1,1,-1
3,1,1,-1,1
4,0,0,1,1


# 5/ Calculate the Discrete Mutual Information between pairs of variables in the dataset

In [4]:
results0 = compute_mi(data=df, mi_func=mutual_information_binary)

Mutual Information between a and b: 0.010833477026353552
Mutual Information between a and x: 0.0005999350832366201
Mutual Information between a and y: 0.00027249720152157115
Mutual Information between b and x: 0.00017487758911443815
Mutual Information between b and y: 2.014195200942116e-05
Mutual Information between x and y: 0.06201744636620489


In [5]:
# This function is expected to be not suitable as all variables seem to be binary
results1 = compute_mi(data=df, mi_func=mutual_information_multiple_discrete)

Mutual Information between a and b: 0.04333390810541421
Mutual Information between a and x: 0.0011998701664732402
Mutual Information between a and y: 0.0005449944030431423
Mutual Information between b and x: 0.0003497551782288763
Mutual Information between b and y: 4.028390401884232e-05
Mutual Information between x and y: 0.06201744636620489


In [6]:
results2 = compute_mi(data=df, mi_func=mutual_info_with_entropy)

Mutual Information between a and b: 0.010833477026353622
Mutual Information between a and x: 0.0005999350832366357
Mutual Information between a and y: 0.00027249720152133783
Mutual Information between b and x: 0.0001748775891143861
Mutual Information between b and y: 2.0141952009389286e-05
Mutual Information between x and y: 0.062017446366204654


# 6/ Compare the results using Scikit-learn functions to calculate mutual information

In [7]:
results3 = compute_mi(data=df, mi_func=mutual_info_score)

Mutual Information between a and b: 0.010833477026354232
Mutual Information between a and x: 0.0005999350832371353
Mutual Information between a and y: 0.0002724972015220595
Mutual Information between b and x: 0.00017487758911494122
Mutual Information between b and y: 2.0141952009777864e-05
Mutual Information between x and y: 0.062017446366205375


In [8]:
results4 = compute_mi(data=df, mi_func=mutual_info_classif)

Mutual Information between a and b: [0.01083348]
Mutual Information between a and x: [0.00059994]
Mutual Information between a and y: [0.0002725]
Mutual Information between b and x: [0.00017488]
Mutual Information between b and y: [2.0141952e-05]
Mutual Information between x and y: [0.06201745]


# 7/ Discussion

The approach of using histogram to estimate discrete mutual information has some characteristics that may limit its efficiency and accuracy for large datasets:

Nested Loops: The function uses nested loops to calculate the mutual information. This means that its time complexity is O(n^2) for the mutual information calculation, where n is the number of unique values in the datasets. For a large number of unique values, this could become a performance bottleneck.

Memory Usage: The function uses histograms for both joint and marginal distributions. The size of the joint histogram is proportional to the product of the number of unique values in both variables. This can increase memory consumption substantially for datasets with many unique values.

Histogram Binning: Using histograms for estimating distributions can introduce errors, especially if the bin sizes aren't chosen appropriately. In this function, the bin sizes are determined based on the number of unique values, which might not always be the best choice. For large datasets with many repeated values, this method could become inaccurate.